In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import multiprocessing as mp
import math
import random
import sys
import time
import requests
import json
import numpy
import importlib
import Clases
from datetime import datetime,date,time, timedelta
from copy import deepcopy

In [46]:
#r2 = requests.get(url='https://aviation-edge.com/v2/public/timetable?key=a24d93-2501aa&iataCode=LIM&type=arrival')
#http://aviation-edge.com/api/public/timetable?key=a24d93-2501aa&iataCode=LIM&type=arrival

In [2]:
with open('ArrivalLima190508.txt') as json_file:  
    data = json.loads(json_file.read().replace("\'", "\""))
    print (data)

#data = r2.json()
print (data)

[{'type': 'arrival', 'status': 'scheduled', 'departure': {'iataCode': 'PIU', 'icaoCode': 'SPUR', 'scheduledTime': '2019-05-08T09:25:00.000'}, 'arrival': {'iataCode': 'LIM', 'icaoCode': 'SPJC', 'scheduledTime': '2019-05-08T11:00:00.000'}, 'airline': {'name': 'Peruvian Airlines', 'iataCode': 'P9', 'icaoCode': 'PVN'}, 'flight': {'number': '411', 'iataNumber': 'P9411', 'icaoNumber': 'PVN411'}}, {'type': 'arrival', 'status': 'scheduled', 'departure': {'iataCode': 'CUZ', 'icaoCode': 'SPZO', 'scheduledTime': '2019-05-08T09:40:00.000'}, 'arrival': {'iataCode': 'LIM', 'icaoCode': 'SPJC', 'scheduledTime': '2019-05-08T11:05:00.000'}, 'airline': {'name': 'Star Peru', 'iataCode': '2I', 'icaoCode': 'SRU'}, 'flight': {'number': '222', 'iataNumber': '2I222', 'icaoNumber': 'SRU222'}, 'codeshared': {'airline': {'name': 'Peruvian Airlines', 'iataCode': 'P9', 'icaoCode': 'PVN'}, 'flight': {'number': '222', 'iataNumber': 'P9222', 'icaoNumber': 'PVN222'}}}, {'type': 'arrival', 'status': 'scheduled', 'depart

In [3]:
data_filtered = list(filter(lambda x : x['status'] != 'landed' and x['status'] != 'cancelled', data))
print(data_filtered)

[{'type': 'arrival', 'status': 'scheduled', 'departure': {'iataCode': 'PIU', 'icaoCode': 'SPUR', 'scheduledTime': '2019-05-08T09:25:00.000'}, 'arrival': {'iataCode': 'LIM', 'icaoCode': 'SPJC', 'scheduledTime': '2019-05-08T11:00:00.000'}, 'airline': {'name': 'Peruvian Airlines', 'iataCode': 'P9', 'icaoCode': 'PVN'}, 'flight': {'number': '411', 'iataNumber': 'P9411', 'icaoNumber': 'PVN411'}}, {'type': 'arrival', 'status': 'scheduled', 'departure': {'iataCode': 'CUZ', 'icaoCode': 'SPZO', 'scheduledTime': '2019-05-08T09:40:00.000'}, 'arrival': {'iataCode': 'LIM', 'icaoCode': 'SPJC', 'scheduledTime': '2019-05-08T11:05:00.000'}, 'airline': {'name': 'Star Peru', 'iataCode': '2I', 'icaoCode': 'SRU'}, 'flight': {'number': '222', 'iataNumber': '2I222', 'icaoNumber': 'SRU222'}, 'codeshared': {'airline': {'name': 'Peruvian Airlines', 'iataCode': 'P9', 'icaoCode': 'PVN'}, 'flight': {'number': '222', 'iataNumber': 'P9222', 'icaoNumber': 'PVN222'}}}, {'type': 'arrival', 'status': 'scheduled', 'depart

In [4]:
class Annealer(object):
    # parámetros
    Tmax = 25000.0
    Tmin = 2.5
    steps = 1000
    updates = 100
    user_exit = False
    max_iter = 1000

    best_state = None
    best_energy = None
    start = None

    def __init__(self):
        """
        Método FIFO para alcanzar una solución mas o menos óptima
        """
        j=0
        for vuelo in listaVuelos:
            asignado = False
            j+=1
            for puerta in listaPuertas:
                if (puerta.insertarVuelo(vuelo)!=-1):
                    asignado = True
                    #printVueloArea(j,vuelo,puerta)
                    if (vuelo.estado == "scheduled"):                        
                        vuelo.addTiempoLlegada(vuelo.tiempoProgramado)
                    else:
                        vuelo.addTiempoLlegada(vuelo.tiempoEstimado)
                    break
                    
            if (asignado):
                continue
                
            for zona in listaZonas:
                if (zona.insertarVuelo(vuelo)!=-1):
                    asignado = True
                    #printVueloArea(j,vuelo,zona)
                    if (vuelo.estado == "scheduled"):                        
                        vuelo.addTiempoLlegada(vuelo.tiempoProgramado)
                    else:
                        vuelo.addTiempoLlegada(vuelo.tiempoEstimado)
                    break

            while(not asignado):
                if (vuelo.estado == "scheduled"):
                    #print (j,vuelo.tiempoProgramado, "GG")
                    vuelo.tiempoProgramado += timedelta(minutes = 30)
                    vuelo.addTiempoLlegada(vuelo.tiempoProgramado)
                else:
                    #print(j,vuelo.tiempoEstimado, "GG")
                    vuelo.tiempoEstimado += timedelta(minutes = 30)
                    vuelo.addTiempoLlegada(vuelo.tiempoEstimado)
                
                for puerta in listaPuertas:
                    if (puerta.insertarVuelo(vuelo)!=-1):
                        asignado = True
                        if (vuelo.estado == "scheduled"):
                            vuelo.tiempoProgramado = vuelo.tiempoSinModificar
                        else:
                            vuelo.tiempoEstimado= vuelo.tiempoSinModificar                            
                        #printVueloArea(j,vuelo,puerta)
                        break

                if (asignado):
                    asignado = True
                    break

                for zona in listaZonas:
                    if (zona.insertarVuelo(vuelo)!=-1):
                        asignado = True
                        if (vuelo.estado == "scheduled"):
                            vuelo.tiempoProgramado = vuelo.tiempoSinModificar
                        else:
                            vuelo.tiempoEstimado= vuelo.tiempoSinModificar 
                        #printVueloArea(j,vuelo,zona)
                        break
        self.state = (listaPuertas, listaZonas)
        best_state = deepcopy(self.state)
        best_energy = self.energy()

    def move(self):
        selector =round(random.random()*2)
        #if (selector == 0 ):
        #mov insercion 
        indiceArea = round(random.random())
        itera =0
        while (True):
            itera +=1
            if(itera >= 1000):
                self.move()
                break
            
            if(indiceArea == 0):
                indicePuertas = round(random.random()*(nPuertas-1))
                #print(indicePuertas)
                area = listaPuertas[indicePuertas]
            else:
                indiceZonas = round(random.random()*(nZonas-1))
                #print(indiceZonas)
                area = listaZonas[indiceZonas]

            if(area.vuelos.cantidad == 0):
                continue
            indiceVuelo = round(random.random()*area.vuelos.cantidad)+1
            cont = 1
            p = area.vuelos.inicio
            while(p is not None):
                asignado = False
                if (p.ocupado and cont == indiceVuelo):
                    #añadir vuelo
                    for puerta in listaPuertas:
                        if (puerta != area and puerta.insertarVuelo(p.vuelo)!=-1):
                            if (vuelo.estado == "scheduled"):                        
                                vuelo.addTiempoLlegada(vuelo.tiempoProgramado)
                            else:
                                vuelo.addTiempoLlegada(vuelo.tiempoEstimado)
                            area.removeVuelo(p)
                            return
                    for zona in listaZonas:
                        if (zona != area and zona.insertarVuelo(p.vuelo)!=-1):
                            if (vuelo.estado == "scheduled"):                        
                                vuelo.addTiempoLlegada(vuelo.tiempoProgramado)
                            else:
                                vuelo.addTiempoLlegada(vuelo.tiempoEstimado)
                            area.removeVuelo(p)
                            return
                    iter2 = 0    
                    while (True):
                        if (vuelo.estado == "scheduled"):
                            vuelo.tiempoProgramado += timedelta(minutes = 10)
                            vuelo.addTiempoLlegada(vuelo.tiempoProgramado)
                        else:
                            vuelo.tiempoEstimado += timedelta(minutes = 10)
                            vuelo.addTiempoLlegada(vuelo.tiempoEstimado)

                        for puerta in listaPuertas:
                            if (puerta.insertarVuelo(vuelo)!=-1):
                                if (vuelo.estado == "scheduled"):
                                    vuelo.tiempoProgramado = vuelo.tiempoSinModificar
                                else:
                                    vuelo.tiempoEstimado= vuelo.tiempoSinModificar
                                area.removeVuelo(p)
                                return

                        for zona in listaZonas:
                            if (zona.insertarVuelo(vuelo)!=-1):
                                if (vuelo.estado == "scheduled"):
                                    vuelo.tiempoProgramado = vuelo.tiempoSinModificar
                                else:
                                    vuelo.tiempoEstimado= vuelo.tiempoSinModificar 
                                
                                area.removeVuelo(p)
                                return 
                        iter2 +=1
                        if (iter2 > 100 ):
                            return

                cont +=1                              
                p=p.sig

        #elif(selector == 1):
        #    pass
        #else:
        #    pass

    def energy(self):
        """Calculate state's energy"""
        costoVuelos = 0
        parCastigo = 900000
        for i in listaVuelos:
            if (i.estado == "scheduled"):
                costoVuelos += (i.tiempoLlegada - i.tiempoProgramado).total_seconds() ** 2
            else: 
                costoVuelos += (i.tiempoLlegada - i.tiempoEstimado).total_seconds() ** 2
                        
        costoAreas = 0
        for puerta in listaPuertas:
            costoAreas += parCastigo * puerta.vuelos.tiempoLibre.total_seconds()   
        
        for zona in listaPuertas:
            costoAreas += zona.vuelos.tiempoLibre.total_seconds()   
        
        #print("Costo: " + str(costoVuelos/3600) + " horas de tardanzas.")
        #print("Costo: "+ str(costoAreas/3600) + " horas no usadas.")
        return costoAreas + costoVuelos

    def anneal(self):
        """
        Minimizar el tiempo de espera de todos los vuelos ya asignados (tiempo real - tiempo programado) 
        y el tiempo sin usar de las puertas del aeropuerto.
        Parameters
        state : an initial arrangement of the system
        Returns
        (state, energy): the best state and energy found.
        """
        step = 0
        if self.Tmin <= 0.0:
            raise Exception('Exponential cooling requires a minimum "\
                "temperature greater than zero.')
        Tfactor = -math.log(self.Tmax / self.Tmin)

        # Note initial state
        T = self.Tmax
        E = self.energy()
        prevState = deepcopy(self.state)
        prevEnergy = E
        self.best_state = deepcopy(self.state)
        self.best_energy = E
        trials, accepts, improves = 0, 0, 0

        # Attempt moves to new states
        while step < self.steps:
            step += 1
            T = self.Tmax * math.exp(Tfactor * step / self.steps)
            self.move()
            E = self.energy()
            dE = E - prevEnergy
            trials += 1
            if dE > 0.0 and math.exp(-dE / T) < random.random():
                # Restore previous state
                self.state = deepcopy(prevState)
                E = prevEnergy
            else:
                # Accept new state and compare to best state
                accepts += 1
                if dE < 0.0:
                    improves += 1
                prevState = deepcopy(self.state)
                prevEnergy = E
                if E < self.best_energy:
                    self.best_state = deepcopy(self.state)
                    self.best_energy = E

        # Return best state and energy
        return self.best_state, self.best_energy

In [5]:
listaVuelos = []
i =0
for flight in data_filtered:
    #print (flight['status'], flight['departure'], flight['arrival'], flight['airline'], flight['flight'])
    try:
        i +=1
        vuelo = Clases.Vuelo()
        jsonDestino = flight ['arrival']
        anho = int(jsonDestino['scheduledTime'][0:4])
        mes = int(jsonDestino['scheduledTime'][6:7])
        dia = int(jsonDestino['scheduledTime'][9:10])
        hora = int(jsonDestino['scheduledTime'][12:13])
        minuto = int(jsonDestino['scheduledTime'][15:16])
        segundo = int(jsonDestino['scheduledTime'][18:19])

        vuelo.addEstado(flight['status'])
        if (vuelo.estado=="scheduled"):
            vuelo.addTiempoEstimado(datetime(year=anho, month=mes, day=dia, \
                                       hour=hora, minute=minuto, second=segundo))
        elif (vuelo.estado=="active"):
            vuelo.addTiempoEstimado(datetime(year=anho, month=mes, day=dia, \
                                       hour=hora, minute=minuto, second=segundo))
            try:
                anho = int(jsonDestino['estimatedTime'][0:4])
                mes = int(jsonDestino['estimatedTime'][6:7])
                dia = int(jsonDestino['estimatedTime'][9:10])
                hora = int(jsonDestino['estimatedTime'][12:13])
                minuto = int(jsonDestino['estimatedTime'][15:16])
                segundo = int(jsonDestino['estimatedTime'][18:19])
                vuelo.addTiempoEstimado(datetime(year=anho, month=mes, day=dia, \
                                           hour=hora, minute=minuto, second=segundo))
            except:
                pass
            
        jsonVuelo = flight['flight']
        vuelo.addNumeroVuelo(jsonVuelo['number'])
        vuelo.addIata(jsonVuelo['iataNumber'])
        print (str(vuelo.tiempoProgramado)+ "  "+ vuelo.numeroVuelo)

        try:
            vuelo.addIcao(jsonVuelo['icaoNumber'])
        except:
            pass

        jsonAerolinea = flight['airline']
        aerolinea =Clases.TAerolinea()
        aerolinea.addIata(jsonAerolinea['iataCode'])
        try:
            aerolinea.addIcao(jsonAerolinea['icaoCode'])
        except:
            pass
        aerolinea.addNombre(jsonAerolinea['name'])

        avion = Clases.Avion()
        avion.addTAerolinea(aerolinea)
        vuelo.addAvion(avion)

        listaVuelos.append(vuelo)
        #print(i)
    except:
        pass

2019-05-08 01:00:00  411
2019-05-08 01:05:00  222
2019-05-08 01:05:00  222
2019-05-08 01:04:00  2012
2019-05-08 01:04:00  2012
2019-05-08 01:05:00  8201
2019-05-08 01:05:00  856
2019-05-08 01:05:00  856
2019-05-08 01:05:00  210
2019-05-08 01:00:00  2170
2019-05-08 01:00:00  2170
2019-05-08 01:03:00  2136
2019-05-08 01:03:00  2136
2019-05-08 01:04:00  2088
2019-05-08 01:04:00  2088
2019-05-08 01:04:00  2313
2019-05-08 01:04:00  2313
2019-05-08 01:09:00  1451
2019-05-08 01:09:00  1451
2019-05-08 01:00:00  2004
2019-05-08 01:00:00  2004
2019-05-08 02:02:00  2022
2019-05-08 02:02:00  2022
2019-05-08 02:02:00  2209
2019-05-08 02:02:00  2209
2019-05-08 02:04:00  5008
2019-05-08 02:04:00  127
2019-05-08 02:04:00  127
2019-05-08 02:00:00  862
2019-05-08 02:00:00  862
2019-05-08 02:03:00  2124
2019-05-08 02:03:00  2124
2019-05-08 02:00:00  3132
2019-05-08 02:03:00  503
2019-05-08 02:08:00  761
2019-05-08 02:00:00  5263
2019-05-08 02:05:00  2251
2019-05-08 02:05:00  2251
2019-05-08 02:05:00  522

In [6]:
importlib.reload(Clases)


<module 'Clases' from 'C:\\Users\\jorge\\Documents\\Jorge\\AlgoRitmoAgogo\\AlgoRitmoAgogo\\Clases.py'>

In [7]:
nPuertas = 20
nZonas = 52
listaZonas = []
listaPuertas = []
for i in range(1,nPuertas+1):
    area2 = Clases.Puerta(2*i,random.random()*79+1, random.random()*59+1,random.random()*499+1,random.random()*499+1,10)
    listaPuertas.append(area2)
    
for i in range(1,nZonas +1):
    area = Clases.Zona(2*i-1,random.random()*79+1, random.random()*59+1,random.random()*499+1, random.random()*499+1)
    listaZonas.append(area)

In [8]:
#print("cantVuelos: "+str(len(listaVuelos)))
ann = Annealer()

In [9]:
x,y = ann.anneal()

print("Puertas")
for i in x[0]:
    i.imprimirLista()
print("Zonas:")
for i in x[1]:
    i.imprimirLista()
print("Resultado: " + str(y))

Puertas
ID: 2
Libre: True | tiempoInicio: 2019-05-18 11:00:00 | tiempoFin: 2019-05-18 14:00:00
Libre: False | tiempoInicio: 2019-05-18 14:00:00 | tiempoFin: 2019-05-18 14:01:00
Libre: True | tiempoInicio: 2019-05-18 14:01:00 | tiempoFin: 2019-05-18 17:01:00
Libre: False | tiempoInicio: 2019-05-18 17:01:00 | tiempoFin: 2019-05-18 17:05:00
Libre: True | tiempoInicio: 2019-05-18 17:05:00 | tiempoFin: 2019-05-18 20:05:00
Libre: True | tiempoInicio: 2019-05-18 20:05:00 | tiempoFin: 2019-05-18 23:05:00
Libre: False | tiempoInicio: 2019-05-18 23:05:00 | tiempoFin: 2019-05-18 23:09:00
Libre: True | tiempoInicio: 2019-05-18 23:09:00 | tiempoFin: 2019-05-19 02:09:00
Libre: False | tiempoInicio: 2019-05-19 02:09:00 | tiempoFin: 2019-05-21 11:59:59
----------------------
ID: 4
Libre: False | tiempoInicio: 2019-05-18 11:00:00 | tiempoFin: 2019-05-18 11:05:00
Libre: True | tiempoInicio: 2019-05-18 11:05:00 | tiempoFin: 2019-05-18 14:05:00
Libre: True | tiempoInicio: 2019-05-18 14:05:00 | tiempoFin: 

In [10]:
def printVueloArea(j,vuelo,puerta):
    
    if (vuelo.estado == "scheduled"):
        print (j,vuelo.tiempoProgramado, puerta.idArea)
    else:
        print(j,vuelo.tiempoEstimado, puerta.idArea)